In [29]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torchcp.classification.scores import THR, APS, Margin, RAPS, SAPS
from torchcp.classification.predictors import SplitPredictor, ClassWisePredictor, ClusterPredictor, WeightedPredictor
import torch.nn.functional as F

In [21]:
# !nvidia-smi

Sun Jan 14 12:09:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     On  | 00000000:65:00.0 Off |                   On |
| N/A   37C    P0              31W / 165W |                  N/A |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

# Device

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Dataset 1: CIFAR10, Model：CNN

In [30]:
class SimpleCNN(torch.nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = torch.nn.Linear(32 * 8 * 8, 256)
        self.fc2 = torch.nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [31]:
# 载入CIFAR-10数据集并进行预处理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./shareddata', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./shareddata', train=False, download=True, transform=transform)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
model = SimpleCNN()
model.to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 设置训练的epoch次数
num_epochs = 20

for epoch in range(num_epochs):
    # 训练模式
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()

Files already downloaded and verified
Files already downloaded and verified


In [32]:
model.eval()
scores = [THR(), Margin(), SAPS(0.5), RAPS(0.01), RAPS(0.1), RAPS(1)]

for i in range(len(scores)):
    # 定义 Conformal Prediction 算法 - ClusterPredictor
    cluster_predictor = ClusterPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClusterPredictor 进行校准
    cluster_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = cluster_predictor.predict(test_instances)
    print("Current Score Function (ClusterPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = cluster_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - SplitPredictor
    split_predictor = SplitPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 SplitPredictor 进行校准
    split_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = split_predictor.predict(test_instances)
    print("Current Score Function (SplitPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = split_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - ClassWisePredictor
    classwise_predictor = ClassWisePredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClassWisePredictor 进行校准
    classwise_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = classwise_predictor.predict(test_instances)
    print("Current Score Function (ClassWisePredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = classwise_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])
    

Current Score Function (ClusterPredictor): THR
0.6491 0.8695
Current Score Function (SplitPredictor): THR
0.6492 0.8697
Current Score Function (ClassWisePredictor): THR
0.6486 0.8829
Current Score Function (ClusterPredictor): Margin
0.6479 0.8727
Current Score Function (SplitPredictor): Margin
0.6479 0.8727
Current Score Function (ClassWisePredictor): Margin
0.6471 0.8865
Current Score Function (ClusterPredictor): SAPS
0.7022 1.1481
Current Score Function (SplitPredictor): SAPS
0.7004 1.1484
Current Score Function (ClassWisePredictor): SAPS
0.6975 1.1453
Current Score Function (ClusterPredictor): RAPS
0.7265 1.3034
Current Score Function (SplitPredictor): RAPS
0.7285 1.3144
Current Score Function (ClassWisePredictor): RAPS
0.7254 1.2969
Current Score Function (ClusterPredictor): RAPS
0.7032 1.1632
Current Score Function (SplitPredictor): RAPS
0.6995 1.1579
Current Score Function (ClassWisePredictor): RAPS
0.7013 1.1586
Current Score Function (ClusterPredictor): RAPS
0.667 0.9695
Curren

# Dataset 2: DTD, Model：CNN

In [35]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [36]:
transform = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = torchvision.datasets.ImageFolder(root='/shareddata/dtd/dtd/images', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/shareddata/dtd/dtd/images', transform=transform)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes=num_classes)
model.to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 设置训练的epoch次数
num_epochs = 32

for epoch in range(num_epochs):
    # 训练模式
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()


In [37]:
model.eval()
scores = [THR(), Margin(), SAPS(0.5), RAPS(0.01), RAPS(0.1), RAPS(1)]

for i in range(len(scores)):
    # 定义 Conformal Prediction 算法 - ClusterPredictor
    cluster_predictor = ClusterPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClusterPredictor 进行校准
    cluster_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = cluster_predictor.predict(test_instances)
    print("Current Score Function (ClusterPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = cluster_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - SplitPredictor
    split_predictor = SplitPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 SplitPredictor 进行校准
    split_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = split_predictor.predict(test_instances)
    print("Current Score Function (SplitPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = split_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - ClassWisePredictor
    classwise_predictor = ClassWisePredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClassWisePredictor 进行校准
    classwise_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = classwise_predictor.predict(test_instances)
    print("Current Score Function (ClassWisePredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = classwise_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

Current Score Function (ClusterPredictor): THR
0.9056737588652483 0.9058510638297872
Current Score Function (SplitPredictor): THR
0.9 0.900177304964539
Current Score Function (ClassWisePredictor): THR
0.9 0.900177304964539
Current Score Function (ClusterPredictor): Margin
0.9037234042553192 0.9039007092198581
Current Score Function (SplitPredictor): Margin
0.9 0.900177304964539
Current Score Function (ClassWisePredictor): Margin
0.9 0.900177304964539
Current Score Function (ClusterPredictor): SAPS
0.9063829787234042 0.9113475177304965
Current Score Function (SplitPredictor): SAPS
0.8907801418439716 0.8948581560283688
Current Score Function (ClassWisePredictor): SAPS
0.9106382978723404 0.9154255319148936
Current Score Function (ClusterPredictor): RAPS
0.9042553191489362 0.926595744680851
Current Score Function (SplitPredictor): RAPS
0.9086879432624113 0.9319148936170213
Current Score Function (ClassWisePredictor): RAPS
0.9047872340425532 0.9271276595744681
Current Score Function (Cluste

# Dataset 3: Oxford Flowers, Model: ResNet18

In [9]:
class FineTunedResNet18(nn.Module):
    def __init__(self, num_classes):
        super(FineTunedResNet18, self).__init__()
        resnet18 = models.resnet18(pretrained=True)
        # 冻结除最后一层之外的所有层
        for param in resnet18.parameters():
            param.requires_grad = False
        # 修改最后一层的输出维度以适应 Oxford Flowers 的类别数
        resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
        self.resnet18 = resnet18

    def forward(self, x):
        return self.resnet18(x)

In [10]:
# Transform for Oxford Flowers test set
transform = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = torchvision.datasets.ImageFolder(root='/shareddata/flowers-102', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/shareddata/flowers-102', transform=transform)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
num_classes = 102  # Oxford Flowers 数据集有 102 个类别
model = FineTunedResNet18(num_classes=num_classes)
model.to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 设置训练的epoch次数
num_epochs = 32

for epoch in range(num_epochs):
    # 训练模式
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()

torch.cuda.empty_cache()

In [11]:
model.eval()
scores = [THR(), Margin(), SAPS(0.5), RAPS(0.01), RAPS(0.1), RAPS(1)]

for i in range(len(scores)):
    # 定义 Conformal Prediction 算法 - ClusterPredictor
    cluster_predictor = ClusterPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClusterPredictor 进行校准
    cluster_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = cluster_predictor.predict(test_instances)
    print("Current Score Function (ClusterPredictor):", type(scores[i]).__name__)
    print(predict_sets)

    # 评估覆盖率和平均集合大小
    result_dict = cluster_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - SplitPredictor
    split_predictor = SplitPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 SplitPredictor 进行校准
    split_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = split_predictor.predict(test_instances)
    print("Current Score Function (SplitPredictor):", type(scores[i]).__name__)
    print(predict_sets)

    # 评估覆盖率和平均集合大小
    result_dict = split_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - ClassWisePredictor
    classwise_predictor = ClassWisePredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClassWisePredictor 进行校准
    classwise_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = classwise_predictor.predict(test_instances)
    print("Current Score Function (ClassWisePredictor):", type(scores[i]).__name__)
    print(predict_sets)

    # 评估覆盖率和平均集合大小
    result_dict = classwise_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

Current Score Function (ClusterPredictor): THR
[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]
0.9000572035629648 1.701397401323854
Current Score Function (SplitPredictor): THR
[[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]
0.9000163438751327 1.701274822260358


/usr/local/lib/python3.10/dist-packages/torchcp/classification/predictors/split.py:54: UserWarning: The shape of scores is 0, which is a invalid scores. To avoid program crash, the threshold is set as torch.inf.
  warnings.warn("The shape of scores is 0, which is a invalid scores. To avoid program crash, the threshold is set as torch.inf.")


Current Score Function (ClassWisePredictor): THR
[[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101], [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4

# Dataset 4:  COIL-20

In [38]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [39]:
transform = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = torchvision.datasets.ImageFolder(root='/data/lab/STA303-Assignment04/data/coil20', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/data/lab/STA303-Assignment04/data/coil20', transform=transform)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes=num_classes)
model.to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 设置训练的epoch次数
num_epochs = 32

for epoch in range(num_epochs):
    # 训练模式
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()

torch.cuda.empty_cache()

In [41]:
model.eval()
scores = [THR(), Margin(), SAPS(0.5), RAPS(0.01), RAPS(0.1), RAPS(1)]
test_iterator = iter(test_dataloader)
for i in range(len(scores)):
    # 定义 Conformal Prediction 算法 - ClusterPredictor
    cluster_predictor = ClusterPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClusterPredictor 进行校准
    cluster_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = cluster_predictor.predict(test_instances)
    print("Current Score Function (ClusterPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = cluster_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - SplitPredictor
    split_predictor = SplitPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 SplitPredictor 进行校准
    split_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据

    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = split_predictor.predict(test_instances)
    print("Current Score Function (SplitPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = split_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - ClassWisePredictor
    classwise_predictor = ClassWisePredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClassWisePredictor 进行校准
    classwise_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据

    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = classwise_predictor.predict(test_instances)
    print("Current Score Function (ClassWisePredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = classwise_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

Current Score Function (ClusterPredictor): THR
0.8965277777777778 0.8965277777777778
Current Score Function (SplitPredictor): THR
0.9 0.9
Current Score Function (ClassWisePredictor): THR
0.9027777777777778 0.9027777777777778
Current Score Function (ClusterPredictor): Margin
0.9034722222222222 0.9034722222222222
Current Score Function (SplitPredictor): Margin
0.9 0.9
Current Score Function (ClassWisePredictor): Margin
0.9027777777777778 0.9027777777777778
Current Score Function (ClusterPredictor): SAPS
0.9041666666666667 0.9041666666666667
Current Score Function (SplitPredictor): SAPS
0.8979166666666667 0.8979166666666667
Current Score Function (ClassWisePredictor): SAPS
0.9166666666666666 0.9166666666666666
Current Score Function (ClusterPredictor): RAPS
0.8979166666666667 0.8979166666666667
Current Score Function (SplitPredictor): RAPS
0.9111111111111111 0.9111111111111111
Current Score Function (ClassWisePredictor): RAPS
0.9152777777777777 0.9152777777777777
Current Score Function (C

# Dataset 5: GTSRB

In [18]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [19]:
# Transform for Oxford Flowers test set
transform = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

train_dataset = torchvision.datasets.ImageFolder(root='/shareddata/gtsrb/GTSRB/Training', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/shareddata/gtsrb/GTSRB/Final_Test', transform=transform)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes=num_classes)
model.to(device)
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 设置训练的epoch次数
num_epochs = 32

for epoch in range(num_epochs):
    # 训练模式
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 梯度清零
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()

torch.cuda.empty_cache()

In [20]:
model.eval()
scores = [THR(), Margin(), SAPS(0.5), RAPS(0.01), RAPS(0.1), RAPS(1)]

for i in range(len(scores)):
    # 定义 Conformal Prediction 算法 - ClusterPredictor
    cluster_predictor = ClusterPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClusterPredictor 进行校准
    cluster_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = cluster_predictor.predict(test_instances)
    print("Current Score Function (ClusterPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = cluster_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - SplitPredictor
    split_predictor = SplitPredictor(score_function=scores[i], model=model)

    # 使用训练数据对 SplitPredictor 进行校准
    split_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = split_predictor.predict(test_instances)
    print("Current Score Function (SplitPredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = split_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

    # 定义 Conformal Prediction 算法 - ClassWisePredictor
    classwise_predictor = ClassWisePredictor(score_function=scores[i], model=model)

    # 使用训练数据对 ClassWisePredictor 进行校准
    classwise_predictor.calibrate(train_dataloader, alpha=0.1)

    # 测试数据
    test_iterator = iter(test_dataloader)
    test_instances, test_labels = next(test_iterator)

    # 进行预测
    predict_sets = classwise_predictor.predict(test_instances)
    print("Current Score Function (ClassWisePredictor):", type(scores[i]).__name__)

    # 评估覆盖率和平均集合大小
    result_dict = classwise_predictor.evaluate(test_dataloader)
    print(result_dict["Coverage_rate"], result_dict["Average_size"])

Current Score Function (ClusterPredictor): THR
0.0 0.5418844022169438
Current Score Function (SplitPredictor): THR
0.0 0.5581947743467933
Current Score Function (ClassWisePredictor): THR
0.0 0.5421219319081552
Current Score Function (ClusterPredictor): Margin
0.0 0.542596991290578
Current Score Function (SplitPredictor): Margin
0.0 0.5574821852731592
Current Score Function (ClassWisePredictor): Margin
0.0 0.5418052256532067
Current Score Function (ClusterPredictor): SAPS
0.002929532858273951 0.9996041171813144
Current Score Function (SplitPredictor): SAPS
0.0032462391132224862 1.0008709422011084
Current Score Function (ClassWisePredictor): SAPS
0.0030878859857482186 0.9995249406175772
Current Score Function (ClusterPredictor): RAPS
0.0035629453681710215 1.0878068091844815
Current Score Function (SplitPredictor): RAPS
0.0038796516231195564 1.0853523357086303
Current Score Function (ClassWisePredictor): RAPS
0.00395882818685669 1.0907363420427554
Current Score Function (ClusterPredictor)